## Compress DICOMs using JPEG 2000 Lossless compression
- take a large 1.9GB DICOM multi-frame image and compress it
- HTJ2K lossless compression

### Requirements
- Transcoding: `nvidia-nvimgcodec-cu12[all] pydicom==3.0.1 cupy-cuda12x>=13.5.0`
- Indexing: `pylibjpeg>=2.0 pylibjpeg-openjpeg>=2.0`

In [0]:
%pip install --quiet numpy==1.26.4 nvidia-nvimgcodec-cu12[all] pydicom==3.0.1 cupy-cuda12x>=13.5.0 pylibjpeg[all]
%pip install mlflow[skinny]

In [0]:
dbutils.library.restartPython()

In [0]:
import yaml
import pprint
compression = "nvImage_HTJ2K"
cfg = yaml.safe_load(open("config.yaml", "r"))
input_path = cfg.get("input_path")
output_path = cfg.get("output_path").replace("{compression}", f"{compression}")
table = cfg.get("table")

pprint.pprint(cfg, indent=4)
print(output_path)

In [0]:
import pydicom
import numpy as np
import cupy as cp
from nvidia import nvimgcodec
from pydicom.encaps import encapsulate
from pydicom.uid import JPEG2000Lossless  # or JPEG2000 for lossy
import time
import torch
print(f"Has GPU: {torch.cuda.is_available()}")
print(f"#GPUs: {torch.cuda.device_count()}")
import os
#os.environ["PYLIBJPEG_OPENJPEG"] = "1"

## Decode


In [0]:
# test decode
import pydicom
import pylibjpeg
ds = pydicom.dcmread(output_path)
pixel_array = ds.pixel_array
shape = pixel_array.shape
is_compressed = ds.file_meta.TransferSyntaxUID.is_compressed
shape, is_compressed

In [0]:
pixel_array.shape, ds.pixel_array_options

In [0]:
from pydicom.dataset import validate_file_meta

# Validate file meta information
try:
    validate_file_meta(ds.file_meta, enforce_standard=True)
    print("File meta information is valid")
except Exception as e:
    print(f"File meta validation error: {e}")